In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

In [7]:
# Load datasets
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')

# Preview data
print(customers.head())
print(products.head())
print(transactions.head())

  CustomerID        CustomerName         Region  SignupDate
0      C0001    Lawrence Carroll  South America  2022-07-10
1      C0002      Elizabeth Lutz           Asia  2022-02-13
2      C0003      Michael Rivera  South America  2024-03-07
3      C0004  Kathleen Rodriguez  South America  2022-10-09
4      C0005         Laura Weber           Asia  2022-08-15
  ProductID              ProductName     Category   Price
0      P001     ActiveWear Biography        Books  169.30
1      P002    ActiveWear Smartwatch  Electronics  346.30
2      P003  ComfortLiving Biography        Books   44.12
3      P004            BookWorld Rug   Home Decor   95.69
4      P005          TechPro T-Shirt     Clothing  429.31
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3       

In [8]:
# Rename the 'Price' column in Products to avoid conflicts
products.rename(columns={'Price': 'ProductPrice'}, inplace=True)

# Merge Transactions with Products
transactions = transactions.merge(products, on='ProductID', how='left')

# Merge Transactions with Customers
data = transactions.merge(customers, on='CustomerID', how='left')

# Verify columns in the merged dataset
print("Columns in merged dataset:", data.columns)

# Check if 'Category' column exists
if 'Category' not in data.columns:
    raise ValueError("Category column is missing in the merged dataset!")



Columns in merged dataset: Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price', 'ProductName', 'Category',
       'ProductPrice', 'CustomerName', 'Region', 'SignupDate'],
      dtype='object')


In [9]:
# Select relevant columns after merging
data = data[[
    'TransactionID', 'CustomerID', 'ProductID', 'TransactionDate', 'Quantity', 
    'TotalValue', 'Price', 'ProductPrice', 'ProductName', 'Category', 
    'CustomerName', 'Region', 'SignupDate'
]]

# Convert date columns to datetime for easier manipulation
data['TransactionDate'] = pd.to_datetime(data['TransactionDate'])
data['SignupDate'] = pd.to_datetime(data['SignupDate'])

# Verify the cleaned dataset
print(data.info())
print(data.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   TransactionID    1000 non-null   object        
 1   CustomerID       1000 non-null   object        
 2   ProductID        1000 non-null   object        
 3   TransactionDate  1000 non-null   datetime64[ns]
 4   Quantity         1000 non-null   int64         
 5   TotalValue       1000 non-null   float64       
 6   Price            1000 non-null   float64       
 7   ProductPrice     1000 non-null   float64       
 8   ProductName      1000 non-null   object        
 9   Category         1000 non-null   object        
 10  CustomerName     1000 non-null   object        
 11  Region           1000 non-null   object        
 12  SignupDate       1000 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(3), int64(1), object(7)
memory usage: 101.7+ KB
None
  Transacti

In [10]:
# Aggregate features for each customer
customer_features = data.groupby('CustomerID').agg({
    'TotalValue': 'sum',                # Total spending
    'Quantity': 'sum',                  # Total quantity purchased
    'ProductID': lambda x: list(x),     # List of products purchased
    'Category': lambda x: list(x),      # List of categories purchased
}).reset_index()

# Add demographic data
customer_features = customer_features.merge(
    data[['CustomerID', 'CustomerName', 'Region', 'SignupDate']].drop_duplicates(),
    on='CustomerID',
    how='left'
)

# Verify the aggregated customer features
print(customer_features.head())


  CustomerID  TotalValue  Quantity  \
0      C0001     3354.52        12   
1      C0002     1862.74        10   
2      C0003     2725.38        14   
3      C0004     5354.88        23   
4      C0005     2034.24         7   

                                          ProductID  \
0                    [P054, P022, P096, P083, P029]   
1                          [P095, P004, P019, P071]   
2                          [P025, P006, P035, P002]   
3  [P049, P053, P038, P025, P097, P024, P008, P077]   
4                                [P025, P039, P012]   

                                            Category        CustomerName  \
0  [Books, Home Decor, Electronics, Electronics, ...    Lawrence Carroll   
1       [Home Decor, Home Decor, Clothing, Clothing]      Elizabeth Lutz   
2    [Home Decor, Home Decor, Clothing, Electronics]      Michael Rivera   
3  [Books, Home Decor, Home Decor, Home Decor, Bo...  Kathleen Rodriguez   
4             [Home Decor, Electronics, Electronics]        

In [14]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# One-hot encode the region
encoder = OneHotEncoder()
region_encoded = encoder.fit_transform(customer_features[['Region']]).toarray()

# Convert product and category lists to string for vectorization
customer_features['ProductID_str'] = customer_features['ProductID'].apply(lambda x: ' '.join(x))
customer_features['Category_str'] = customer_features['Category'].apply(lambda x: ' '.join(x))

# Vectorize product and category features
vectorizer = CountVectorizer()
product_vectorized = vectorizer.fit_transform(customer_features['ProductID_str'])
category_vectorized = vectorizer.fit_transform(customer_features['Category_str'])

# Normalize numerical features
scaler = MinMaxScaler()
numerical_features = scaler.fit_transform(customer_features[['TotalValue', 'Quantity']])

# Combine all features into a single matrix
final_features = np.hstack([
    numerical_features,
    region_encoded,
    product_vectorized.toarray(),
    category_vectorized.toarray()
])


In [15]:
# Calculate the cosine similarity between customers based on the final feature matrix
similarity_matrix = cosine_similarity(final_features)

# Convert the similarity matrix to a DataFrame for easier viewing
similarity_df = pd.DataFrame(similarity_matrix, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])

# Display the similarity matrix for the first few customers
print(similarity_df.head())


CustomerID     C0001     C0002     C0003     C0004     C0005     C0006  \
CustomerID                                                               
C0001       1.000000  0.235298  0.493659  0.602047  0.599819  0.350770   
C0002       0.235298  1.000000  0.629403  0.465386  0.387502  0.425560   
C0003       0.493659  0.629403  1.000000  0.658998  0.575314  0.456652   
C0004       0.602047  0.465386  0.658998  1.000000  0.554278  0.601118   
C0005       0.599819  0.387502  0.575314  0.554278  1.000000  0.192508   

CustomerID     C0007     C0008     C0009     C0010  ...     C0191     C0192  \
CustomerID                                          ...                       
C0001       0.674481  0.566430  0.237775  0.070637  ...  0.599082  0.572590   
C0002       0.388197  0.708939  0.324685  0.380651  ...  0.007724  0.152257   
C0003       0.495857  0.755036  0.259976  0.209777  ...  0.187413  0.318645   
C0004       0.506430  0.715236  0.108761  0.174213  ...  0.513855  0.388319   
C0005  

In [16]:
# Create a dictionary to store lookalikes for each customer
lookalikes = {}

# For each customer, find the top 3 lookalikes (excluding themselves)
top_n = 3  # Number of lookalikes you want to identify
for customer_id in similarity_df.index:
    # Get the similarity scores for the customer
    similar_customers = similarity_df[customer_id].sort_values(ascending=False)
    
    # Drop the customer itself and select the top N similar customers
    top_lookalikes = similar_customers.drop(customer_id).head(top_n).index.tolist()
    
    # Store the lookalikes
    lookalikes[customer_id] = top_lookalikes

# Display the lookalikes dictionary
print(lookalikes)


{'C0001': ['C0190', 'C0069', 'C0120'], 'C0002': ['C0134', 'C0133', 'C0178'], 'C0003': ['C0031', 'C0181', 'C0163'], 'C0004': ['C0065', 'C0041', 'C0047'], 'C0005': ['C0096', 'C0162', 'C0007'], 'C0006': ['C0071', 'C0139', 'C0147'], 'C0007': ['C0140', 'C0162', 'C0045'], 'C0008': ['C0059', 'C0162', 'C0104'], 'C0009': ['C0062', 'C0105', 'C0056'], 'C0010': ['C0099', 'C0034', 'C0092'], 'C0011': ['C0171', 'C0126', 'C0087'], 'C0012': ['C0065', 'C0152', 'C0076'], 'C0013': ['C0107', 'C0049', 'C0099'], 'C0014': ['C0128', 'C0089', 'C0060'], 'C0015': ['C0036', 'C0065', 'C0038'], 'C0016': ['C0183', 'C0072', 'C0192'], 'C0017': ['C0075', 'C0081', 'C0104'], 'C0018': ['C0064', 'C0068', 'C0087'], 'C0019': ['C0064', 'C0191', 'C0070'], 'C0020': ['C0026', 'C0050', 'C0007'], 'C0021': ['C0045', 'C0075', 'C0101'], 'C0022': ['C0175', 'C0145', 'C0141'], 'C0023': ['C0191', 'C0068', 'C0019'], 'C0024': ['C0116', 'C0194', 'C0175'], 'C0025': ['C0104', 'C0160', 'C0065'], 'C0026': ['C0197', 'C0098', 'C0149'], 'C0027': ['

In [17]:
import csv

# Open the file Lookalike.csv in write mode
with open('Lookalike.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header row: ["CustomerID", "Lookalikes"]
    writer.writerow(['CustomerID', 'Lookalikes'])
    
    # Write the lookalikes data
    for customer_id, lookalike_list in lookalikes.items():
        # Join the lookalike list into a string, separated by commas
        writer.writerow([customer_id, ', '.join(map(str, lookalike_list))])
        
print("Lookalike.csv has been successfully created.")


Lookalike.csv has been successfully created.


In [18]:
# Load the Lookalike.csv to verify the output
lookalikes_df = pd.read_csv('Lookalike.csv')
print(lookalikes_df.head())


  CustomerID           Lookalikes
0      C0001  C0190, C0069, C0120
1      C0002  C0134, C0133, C0178
2      C0003  C0031, C0181, C0163
3      C0004  C0065, C0041, C0047
4      C0005  C0096, C0162, C0007
